# TFLite Analyzer

In [ ]:
!pip install tflite-support

## Extraction Code

In [ ]:
import os
import flatbuffers
import tensorflow as tf
from tensorflow.lite.python import schema_py_generated as tflite_schema

def analyze_tflite(file_path):
    # Load the raw file
    with open(file_path, "rb") as f:
        data = f.read()
    size_total = len(data)

    # Parse the FlatBuffer model
    model = tflite_schema.Model.GetRootAsModel(data, 0)

    print(f"=== TFLite Model Analysis: {os.path.basename(file_path)} ===")
    print(f"Total file size: {size_total:,} bytes")

    # Subgraphs
    num_subgraphs = model.SubgraphsLength()
    print(f"Number of subgraphs: {num_subgraphs}")

    # Buffers
    num_buffers = model.BuffersLength()
    buffer_sizes = []
    for i in range(num_buffers):
        buf = model.Buffers(i)
        buf_data = buf.DataAsNumpy() if buf.DataLength() > 0 else b''
        buffer_sizes.append(len(buf_data))
    total_buffer_size = sum(buffer_sizes)
    print(f"Buffers: {num_buffers} (total {total_buffer_size:,} bytes)")
    if buffer_sizes:
      print(f"  Largest buffer: {max(buffer_sizes):,} bytes")


    # Operator codes
    # Get all items from tflite_schema.BuiltinOperator()
    builtin_operators = []
    for name, value in tflite_schema.BuiltinOperator.__dict__.items():
        if isinstance(value, int):
            builtin_operators.append(name)

    num_opcodes = model.OperatorCodesLength()
    print(f"Operator codes: {num_opcodes}")
    print("Opcodes:")
    for i in range(num_opcodes):
        opcode = model.OperatorCodes(i)
        builtin_code = opcode.BuiltinCode()
        # Use the BuiltinOperator enum to get the name
        print(f"  - ({builtin_code}) {builtin_operators[builtin_code]}")


    # Estimate sizes
    # FlatBuffer is a packed structure, so we can’t get exact byte offsets easily
    # But we can give relative storage cost
    print("\n--- Estimated Component Sizes ---")
    print(f"Buffers (weights/constants): {total_buffer_size:,} bytes "
          f"({100*total_buffer_size/size_total:.2f} % of file)")
    print(f"Other metadata & structure: {size_total-total_buffer_size:,} bytes "
          f"({100*(size_total-total_buffer_size)/size_total:.2f} % of file)")

    return {
        "total": size_total,
        "buffers": total_buffer_size,
        "other": size_total - total_buffer_size,
        "num_buffers": num_buffers,
        "num_opcodes": num_opcodes,
        "num_subgraphs": num_subgraphs
    }

## Dynamic Rang PTQ

In [ ]:
# Example usage:
report = analyze_tflite("/content/teacher_model_ptq.tflite")
print(report)

=== TFLite Model Analysis: teacher_model_ptq.tflite ===
Total file size: 764,504 bytes
Number of subgraphs: 1
Buffers: 32 (total 753,032 bytes)
  Largest buffer: 688,128 bytes
Operator codes: 7
Opcodes:
  - (70) EXPAND_DIMS
  - (3) CONV_2D
  - (18) MUL
  - (0) ADD
  - (22) RESHAPE
  - (9) FULLY_CONNECTED
  - (14) LOGISTIC

--- Estimated Component Sizes ---
Buffers (weights/constants): 753,032 bytes (98.50 % of file)
Other metadata & structure: 11,472 bytes (1.50 % of file)
{'total': 764504, 'buffers': 753032, 'other': 11472, 'num_buffers': 32, 'num_opcodes': 7, 'num_subgraphs': 1}


## Integer-Only PTQ

In [ ]:
report_int = analyze_tflite("/content/teacher_model_ptq_int.tflite")
print(report_int)

=== TFLite Model Analysis: teacher_model_ptq_int.tflite ===
Total file size: 768,704 bytes
Number of subgraphs: 1
Buffers: 34 (total 749,192 bytes)
  Largest buffer: 688,128 bytes
Operator codes: 8
Opcodes:
  - (114) QUANTIZE
  - (70) EXPAND_DIMS
  - (3) CONV_2D
  - (18) MUL
  - (0) ADD
  - (22) RESHAPE
  - (9) FULLY_CONNECTED
  - (14) LOGISTIC

--- Estimated Component Sizes ---
Buffers (weights/constants): 749,192 bytes (97.46 % of file)
Other metadata & structure: 19,512 bytes (2.54 % of file)
{'total': 768704, 'buffers': 749192, 'other': 19512, 'num_buffers': 34, 'num_opcodes': 8, 'num_subgraphs': 1}
